## Contribution analysis in the foreground system

### Massimo Pizzol, 2024

In [1]:
# Note: must be environment bw2
import brightway2 as bw
import pandas as pd
import numpy as np

Setting up the project with databases, a LCIA method, and some activities

In [2]:
bw.projects.set_current("advlca23") # my project with ei3.9 conseq

In [45]:
# a method for LCIA
mymethod = ('ReCiPe 2016 v1.03, midpoint (H)', 'climate change', 'global warming potential (GWP1000)')

In [34]:
# Make a list of activities from ecoinvent, chosen randomly
myacts = [bw.Database('ecoinvent 3.9 conseq').random() for _ in range(5)]
myacts

['treatment of wastewater from hard fibreboard production, wastewater treatment' (cubic meter, RER, None),
 'market for 6-benzyladenine' (kilogram, GLO, None),
 'market for electricity, medium voltage' (kilowatt hour, AT, None),
 'market for cobalt carbonate' (kilogram, GLO, None),
 'aluminium chloride production' (kilogram, GLO, None)]

### One static score

In [46]:
# Forthe first activity in the list
myact = bw.Database(myacts[1]['database']).get(myacts[1]['code'])
print(myact)
functional_unit = {myact: 1} 
lca = bw.LCA(functional_unit, mymethod)
lca.lci()
lca.lcia()
print(lca.score)

'market for 6-benzyladenine' (kilogram, GLO, None)
8.716756507694592


Function to get LCA results

In [23]:
def dolcacalc(myact, mydemand, mymethod):
    my_fu = {myact: mydemand} 
    lca = bw.LCA(my_fu, mymethod)
    lca.lci()
    lca.lcia()
    return lca.score

# For WtT
def getLCAresults(list_acts, mymethod):
    
    all_activities = []
    results = []
    for a in list_acts:
        act = bw.Database(a[0]).get(a[1])
        all_activities.append(act['name'])
        results.append(dolcacalc(act,1,mymethod)) # 1 stays for one unit of each process
        #print(act['name'])
     
    results_dict = dict(zip(all_activities, results))
    
    return results_dict

# Contribution Analysis

One might want to know how much each exchange of an activity contributes to the total impact of such "parent" activity.

What I do here is iterating through the exchanges of the "parent" activity and calculate impact scores for each exchange as if this was the functional unit, using the amount used in input to the "parent" activity

The tings to notice is that a different approach is used for biosphere and technosphere exchanges

In [36]:
ca_dict = {}

for act in myacts:
    
    exc_list = []
    contr_list = []

    for exc in list(act.exchanges()):
        
        if exc['type'] == 'biosphere':
            
            col = lca.activity_dict[exc['output']] # find column index of A matrix for the activity
            row = lca.biosphere_dict[exc['input']] # find row index of B matrix for the exchange
            contr_score = lca.biosphere_matrix[row,col] * lca.characterization_matrix[row,row]
            contr_list.append((exc['input'], exc['type'], exc['amount'], contr_score))
            
        elif exc['type'] == 'substitution':
            
            contr_score = dolcacalc(bw.Database(exc['input'][0]).get(exc['input'][1]), exc['amount'], mymethod)
            contr_list.append((exc['input'], exc['type'], exc['amount'], -contr_score))
            
        else:
            
            contr_score = dolcacalc(bw.Database(exc['input'][0]).get(exc['input'][1]), exc['amount'], mymethod)
            contr_list.append((exc['input'], exc['type'], exc['amount'], contr_score))
        
    ca_dict[act['code']] =  contr_list

### We can check one activity

Using some nice formatting

In [42]:
df = pd.DataFrame(ca_dict[myacts[1]['code']], columns = ['input','type','amount','contribution'])
df

,input,type,amount,contribution
0,"(ecoinvent 3.9 conseq, db9a71a0edf13c89a31140e...",production,1.0000,8.716757
1,"(ecoinvent 3.9 conseq, db232fc4d8484ff70f90aaf...",technosphere,1.0000,8.659267
2,"(ecoinvent 3.9 conseq, f005944b534083aff9e9922...",technosphere,0.3091,0.017582
3,"(ecoinvent 3.9 conseq, 9ef3e3988eccdfcbf356d2d...",technosphere,0.0246,0.001317
4,"(ecoinvent 3.9 conseq, 70f7992d941cafc4dd70f28...",technosphere,0.2088,0.032328
5,"(ecoinvent 3.9 conseq, 390ba2d0a2107982360b357...",technosphere,0.5990,0.006263


Let's check it gives the right result, and the sum of all the exchanges equals the impact of the production activity

It's not perfectly close. I guess due to rounding errors? The difference is very very small.

In [43]:
print(df.loc[df['type'] == 'production']['contribution'].sum())
print(df.loc[df['type'] != 'production']['contribution'].sum())

8.716756507694592
8.716756507406608


Here below I calculate the contribution in percentage. 

In [44]:
df['%_contribution'] = 100 * df['contribution'] / df.loc[df['type'] == 'production']['contribution'].sum()
df

,input,type,amount,contribution,%_contribution
0,"(ecoinvent 3.9 conseq, db9a71a0edf13c89a31140e...",production,1.0000,8.716757,100.000000
1,"(ecoinvent 3.9 conseq, db232fc4d8484ff70f90aaf...",technosphere,1.0000,8.659267,99.340477
2,"(ecoinvent 3.9 conseq, f005944b534083aff9e9922...",technosphere,0.3091,0.017582,0.201708
3,"(ecoinvent 3.9 conseq, 9ef3e3988eccdfcbf356d2d...",technosphere,0.0246,0.001317,0.015104
4,"(ecoinvent 3.9 conseq, 70f7992d941cafc4dd70f28...",technosphere,0.2088,0.032328,0.370867
5,"(ecoinvent 3.9 conseq, 390ba2d0a2107982360b357...",technosphere,0.5990,0.006263,0.071844
